# Types of Recommendation Saystem
# #  1  Context based
# #  2  Collaborate filtering
# #  3  Hybrid


# Flow  
## dataset -> PreProcessing ->

In [3]:
import pandas as pd
import numpy as np

# Load DataSET

In [5]:
data_credit = pd.read_csv("tmdb_5000_credits.csv")
data_credit.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [6]:
data_movies = pd.read_csv("tmdb_5000_Movies.csv")
data_movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [7]:
# data_movies["genres"].values

In [8]:
# data_movies["keywords"].values

In [9]:
# data_movies["spoken_languages"].values

###  Merge both datasets

In [11]:
movies = data_movies.merge(data_credit , on = "title")
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [12]:
# movies.shape

# #  select colums as we need

In [14]:
# id , genres , keywords , 	title	, overview ,  ,cast	, crew

In [15]:
movies = movies[["id" , "title" , "genres" , "cast" , "crew" ,"keywords" , "overview" ]]
# movies.head(3)

In [16]:
movies.isnull().sum()

id          0
title       0
genres      0
cast        0
crew        0
keywords    0
overview    3
dtype: int64

###  Drop null values 

In [18]:
movies.dropna(inplace = True)

In [19]:
# movies.duplicated().sum()

In [20]:
# movies.iloc[0].genres

## convert  genres from this
###### '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
## To This
###### ["Action" , Advanture , "Fantasy" , "Science Fiction"]

### write a funtion 

In [23]:
#  ast.literal_eval covert string into list
import ast
def convert(obj):
    list = []
    for i in ast.literal_eval(obj) :
        list.append(i["name"])
    return list

## Convert genres

In [25]:
movies["genres"] = movies["genres"].apply(convert)
# movies.head(1)

## convert Keywords

In [27]:
movies["keywords"] = movies["keywords"].apply(convert)
# movies.head(1)

In [28]:
# movies.iloc[0].cast

### we need only three character name from cast 

In [30]:
def convert2(obj):
    list = []
    counter = 0
    for i in ast.literal_eval(obj) :
        if counter != 3:
            list.append(i["name"])
            counter+=1
        else:
            break
    return list

# convert cast

In [32]:
movies["cast"] = movies["cast"].apply(convert2)
# movies.head(1)

## Fetch Director from crew because we need only director

In [34]:
# movies.iloc[0].crew

In [35]:
def fetch_director(obj):
    list = []
    
    for i in ast.literal_eval(obj):
        if i["job"] == "Director":
            list.append(i["name"])
            break
    return list

In [36]:
movies["crew"] = movies["crew"].apply(fetch_director)
# movies.head(1)

In [37]:
# movies.iloc[0].overview

### Overview is String so firstly we convert this into list

In [39]:
movies["overview"] = movies["overview"].apply(lambda x: x.split())
# movies.head(1)

## remove a space from string 

In [41]:
movies["genres"] = movies["genres"].apply(lambda x: [i.replace(" " , "") for i in x])
movies["crew"] = movies["crew"].apply(lambda x: [i.replace(" " , "") for i in x])
movies["cast"] = movies["cast"].apply(lambda x: [i.replace(" " , "") for i in x])
movies["keywords"] = movies["keywords"].apply(lambda x: [i.replace(" " , "") for i in x])

In [42]:
# movies.head(1) 

## Create another column on the name of "tags" and add "genres","cast","overview","keywords","crew" in "tags" column

In [44]:
movies["tags"] = movies["overview"] + movies["genres"] + movies["keywords"] + movies["cast"] + movies["crew"]
# movies.head(1)

## create new data set 

In [46]:
movies = movies[["id" , "title" , "tags"]]
# movies.head(3)

## convert "tags" column from list to string 

In [48]:
movies["tags"] = movies["tags"].apply(lambda x:" ".join(x))
# movies.head(3)

### convert tags in  lower case this is recommended

In [50]:
movies["tags"] = movies["tags"].apply(lambda x:x.lower())
movies.head(3)

,id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...


### We Need to stem "tags" data. for example {"love" , "lover" , "loving"}
### All mean is same so we convert all in same ["love" , "love" , "love"].

In [52]:
import nltk

In [53]:
from nltk.stem.porter import PorterStemmer

In [54]:
ps = PorterStemmer()

In [55]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [56]:
movies["tags"] = movies["tags"].apply(stem)
movies.head(3)

,id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...


#  Text Vectorization on "tags"
####  is the process of converting text data (such as words or sentences) into numerical vectors that machine learning 
#### models can understand and work with
## Common Text Vectorization Methods
# # # 1 : Bag of Words (BoW)
# # # 2 : Term Frequency - Inverse Document Frequency (TF-IDF)
# # # 3 : Word Embeddings (e.g., Word2Vec, GloVe, FastText)
# # # 4 : Document Embeddings (e.g., Doc2Vec)
# # # 5 : N-Grams 
# # # 6 : Tokenization (with Transformers)

### WE USE FIRST BoW

In [59]:
from sklearn.feature_extraction.text import CountVectorizer

In [60]:
cv = CountVectorizer(max_features = 5000 , stop_words = "english")

In [61]:
vectors = cv.fit_transform(movies["tags"]).toarray()
# vectors

In [62]:
# vectors[0]

In [63]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

## we find a distance between two movies using cosine_similarity
## how many this similar to each other

In [65]:
from sklearn.metrics.pairwise import cosine_similarity

In [66]:
similarity = cosine_similarity(vectors)
# similarity
# similarity[0]

In [67]:
# sorted(list(enumerate(similarity[1] )) ,reverse = True , key = lambda x:x[1])[1:6]

In [68]:
def recommend(movie):
    # find movie index
    movies_index = movies[movies["title"] == movie].index[0]
    # find similarity
    distance = similarity[movies_index]
    movies_list = sorted(list(enumerate(distance)),reverse =True ,key =lambda x:x[1])[1:6]

    for i in movies_list:
        print(movies.iloc[i[0]].title)

In [69]:
recommend("Batman Begins")

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf
